(Speaking notes for a talk on database architecture)

* There are just three properties intrinsic to a database.
  * Persistent insert --- You can push data into the database, and expect it to still be there later.
  * Persistent read --- You can read data out of the database, and expect it to return the same value as later.
  * Delete --- You can delete data from the database.
  
  
* Not included in these properties:
  * Transactions --- transactions are thought of as a common database feature. But they are not an intrinsic thing!


* Classical transactional databases.
  * ACID --- atomic, consistent, isolated, durable.
  * Originally invented for the online transaction processing, or OLTP, use case.
  * That explains a lot about why these properties, particularly the second one, are important.


* External data model
  * Data is modeled using tables.
  * SQL --- Structured Query Language is provided as an abstraction.
      * SQL queries reconstruct data of interest by performing joins between tables.
      * SQL is what is called a "declarative query language". They are designed to look kind of like English sentences.
  * Why do you need SQL at all? So that database writing skill is portable between databases.
  * Why does it need to be declarative? To hide the details of database internals (the internal data model) from you.
  * Why is this useful? Because queries that you write against databases are not immediately executed; instead they are pushed through a query optimizer, which finds ways to speed your query up by using its knowledge of the underlying hardware and software layers.
  * SQL says what you want, not how you want. The alternative is imperative - like writing a program.
  * Unfortunately this abstraction is a bit leaky, due to vendors implementing things as they like them.
  
* Example transactional databases
  * SQL databases are not the same, but their user-facing design is the same.
  * SQLite
      * File-based.
      * No concurrency, only one in-flight request at a time.
      * Transactions.
      * Limited SQL implementation.
      * Open-source.
  * Postgres
      * Complicated.
      * Concurrent.
      * Transactions.
      * Broad SQL implementation.
      * Open-source.
  * OracleDB?
      * Complicated.
      * Concurrent.
      * Transactions.
      * Broad SQL implementation.
      * Closed-source.




* Simplest possible database: append to a file.
* Simple log-structured database.
* Add log merges.
* This is Bitcask!
* Why are log files good? Append operation is fast.


* More complicated arch: SSLTables.
* Use a memcache which supports random order, sorted iteration to write log files that are key-sorted.
* Log files are now much faster to merge.


* Final architecture: B-trees.
* B-trees sort data into pages, which are accessed from previous pages.
* B-trees involve random seek-writes, but do not necessitate a log merge process.
* So more predictable performance, but worse median write performance.

* NoSQL databases use different data models.